# 2 - APIs Geocode and Foursquare

### Import Libraries

In [41]:
import requests 
import json
from pandas import json_normalize
import pyjsonviewer
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator
import geopandas as gdp
import shapely

from pymongo import MongoClient

import sys
sys.path.append('../')
import src.api_functions as af
import src.offices_functions as of


import warnings
warnings.filterwarnings('ignore')

- NEED 1: The CEO is vegan = 1 --> vegan restaurants

- NEED 2: The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away = 87 --> vet or pet hairdresser

- NEED 3: Everyone in the company is between 25 and 40, give them some place to go party = 87  --> pubs and clubs

- NEED 4: 30% of the company staff have at least 1 child = 26 --> nurseries or schools

- NEED 5: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. = 20 --> gaming companies around

- NEED 6: Account managers need to travel a lot = 20 --> airports or train stations

- NEED 7: Developers like to be near successful tech startups that have raised at least 1 Million dollars = 15 (Front and Back) --> succesfull tech companies around

In [2]:
load_dotenv()

True

In [3]:
client_id = os.getenv("token1")
client_secret = os.getenv("token2")

### We get the coordenates the selected cities

In [4]:
df_affinity_companies = pd.read_csv("affinity_companies.csv")
df_affinity_companies_top_cities =  df_affinity_companies.groupby(["city", "country_code" ] ).size().to_frame(name = "companies_count").reset_index()
df_top2_cities = df_affinity_companies_top_cities.nlargest(2,"companies_count")
df_top2_cities

,city,country_code,companies_count
195,San Francisco,USA,39
154,New York,USA,27


### San Francisco address

In [5]:
df_affinity_companies_subset = df_affinity_companies[["name", "address1", "city", "latitude", "longitude", "country_code"]]
df_san_francisco_address = df_affinity_companies_subset[(df_affinity_companies_subset["city"] == "San Francisco") & (df_affinity_companies_subset["country_code"] == "USA") & (df_affinity_companies_subset["name"] == "PlayFirst")]

In [6]:
df_san_francisco_address

,name,address1,city,latitude,longitude,country_code
246,PlayFirst,160 Spear St,San Francisco,37.790346,-122.40185,USA


### New York address

In [7]:
df_affinity_companies_subset = df_affinity_companies[["name", "address1", "city", "latitude", "longitude", "country_code"]]
df_new_york_address = df_affinity_companies_subset[(df_affinity_companies_subset["city"] == "New York") & (df_affinity_companies_subset["country_code"] == "USA") & (df_affinity_companies_subset["name"] == "Major League Gaming")]

In [8]:
df_new_york_address

,name,address1,city,latitude,longitude,country_code
110,Major League Gaming,3 Park Avenue,New York,40.752672,-73.97593,USA


In [9]:
top_cities_address = ["160 Spear St San Francisco USA", "3 Park Avenue New York USA"]

In [10]:
coords = af.getCoordinates(top_cities_address)

In [11]:
for k, v in coords.items():
    print(v.keys())

dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])


In [12]:
print(coords.items())

dict_items([('160 Spear St San Francisco USA', {'standard': {'stnumber': '160', 'addresst': 'SPEAR ST', 'postal': '94105-1542', 'region': 'CA', 'prov': 'US', 'city': 'San Francisco', 'countryname': 'United States of America', 'confidence': '1.00'}, 'longt': '-122.39312', 'alt': {}, 'elevation': {}, 'latt': '37.79179'}), ('3 Park Avenue New York USA', {'standard': {'stnumber': '3', 'addresst': 'Park Ave', 'postal': '11201', 'region': 'NY', 'zip': '11201', 'prov': 'US', 'city': 'New York', 'countryname': 'United States of America', 'confidence': '0.9'}, 'longt': '-73.98037', 'alt': {}, 'elevation': {}, 'latt': '40.69651'})])


In [13]:
coordinates = af.actualCoordinates(coords)

In [14]:
coordinates

{'160 Spear St San Francisco USA': [-122.39312, 37.79179],
 '3 Park Avenue New York USA': [-73.98037, 40.69651]}

### FOURSQUARE API CALLING

We are trying to meet now requirements in bold:

#### - NEED 1: The CEO is vegan = 1 --> vegan restaurants

#### - NEED 2: The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away = 87 --> vet or pet hairdresser

#### - NEED 3: Everyone in the company is between 25 and 40, give them some place to go party = 87  --> pubs and clubs

#### - NEED 4: 30% of the company staff have at least 1 child = 26 --> nurseries or schools

- NEED 5: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. = 20 --> gaming companies around

#### - NEED 6: Account managers need to travel a lot = 20 --> airports or train stations

- NEED 7: Developers like to be near successful tech startups that have raised at least 1 Million dollars = 15 (Front and Back) --> succesfull tech companies around

In [15]:
queries = ["Nursery", "Preschool", "Train Station", "Airport", "Pub", "Club", "Vet", "Pet Grooming", "vegan restaurant"]

In [16]:
url = "https://api.foursquare.com/v2/venues/explore"

In [18]:
San_Francisco_USA = af.foursquareQueries(coordinates["160 Spear St San Francisco USA"][1],coordinates["160 Spear St San Francisco USA"][0], url, *queries)

In [150]:
# San_Francisco_USA

In [21]:
New_York_USA = af.foursquareQueries(coordinates["3 Park Avenue New York USA"][1],coordinates["3 Park Avenue New York USA"][0], url, *queries)

###  Foursquare to Mongo

In [ ]:
# with open('data.json', 'w') as f:
#     json.dump(resp, f)

In [ ]:
# pyjsonviewer.view_data(json_data=resp)

In [22]:
San_Francisco_USA2 = af.responseGroups(San_Francisco_USA)

In [23]:
New_York_USA2 = af.responseGroups(New_York_USA)

In [151]:
# New_York_USA2

In [ ]:
# def getFromDict(diccionario, mapa):
#     return reduce(operator.getitem,mapa,diccionario)

In [ ]:
# def type_point(lista):
#     return {"type": "Point", 
#             "coordinates": lista }

### Dictionaries to dataframe

In [26]:
df_san_francisco_usa = af.toDataframe(*San_Francisco_USA2) 

In [27]:
df_san_francisco_usa.head(1)

,name,latitud,longitud,category
0,Bright Horizons at Spear Street,37.790787,-122.392536,"[{'id': '4f4532974b9074f6e4fb0104', 'name': 'D..."


In [113]:
# Get key name from category

df_san_francisco_usa_final = json_normalize(df_san_francisco_usa["category"])

In [114]:
df_san_francisco_usa_final.head(2)

,0
0,"{'id': '4f4532974b9074f6e4fb0104', 'name': 'Da..."
1,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr..."


In [115]:
df_san_francisco_usa_final.rename(columns = {0:'category_all'}, inplace = True)

In [116]:
# df_san_francisco_usa_final.head()

In [117]:
column_dict_keys = ["name"]

In [118]:
of.getFromDictColumn("category_all", column_dict_keys, df_san_francisco_usa_final)

,category_all,name
0,"{'id': '4f4532974b9074f6e4fb0104', 'name': 'Da...",Daycare
1,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
2,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
3,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
4,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool


In [125]:
df_san_francisco_usa_final2 = df_san_francisco_usa_final.drop(columns=['category_all'])
df_san_francisco_usa2 = df_san_francisco_usa.drop(columns=['category'])

In [126]:
df_san_francisco_usa_final2.rename(columns = {"name":"category"}, inplace = True)
df_san_francisco_usa_final2.head(2)

,category
0,Daycare
1,Preschool


In [127]:
df_san_francisco_usa_final3 = pd.concat([df_san_francisco_usa2, df_san_francisco_usa_final2], axis=1)

In [131]:
df_san_francisco_usa_final3.head(2)

,name,latitud,longitud,category
0,Bright Horizons at Spear Street,37.790787,-122.392536,Daycare
1,St. Mary's International Preschool,37.795976,-122.404742,Preschool


In [129]:
df_new_york_usa = af.toDataframe(*New_York_USA2)

In [130]:
df_new_york_usa.head(1)

,name,latitud,longitud,category
0,Williamsburg Northside Preschool,40.71657,-73.958664,"[{'id': '52e81612bcbc57f1066b7a45', 'name': 'P..."


In [141]:
# Get key name from category

df_new_york_usa_final = json_normalize(df_new_york_usa["category"])

In [142]:
df_new_york_usa_final.head(2)

,0
0,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr..."
1,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr..."


In [143]:
df_new_york_usa_final.rename(columns = {0:'category_all'}, inplace = True)

In [144]:
of.getFromDictColumn("category_all", column_dict_keys, df_new_york_usa_final)

,category_all,name
0,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
1,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
2,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
3,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool
4,"{'id': '52e81612bcbc57f1066b7a45', 'name': 'Pr...",Preschool


In [145]:
df_new_york_usa_final2 = df_new_york_usa_final.drop(columns=['category_all'])
df_new_york_usa2 = df_new_york_usa.drop(columns=['category'])

In [146]:
df_new_york_usa_final2.rename(columns = {"name":"category"}, inplace = True)
# df_new_york_usa_final2.head(2)

In [147]:
df_new_york_usa_final3 = pd.concat([df_new_york_usa2, df_new_york_usa_final2], axis=1)
df_new_york_usa_final3.head(2)

,name,latitud,longitud,category
0,Williamsburg Northside Preschool,40.716570,-73.958664,Preschool
1,Smiling Faces Christian Day Care,40.693304,-73.970705,Preschool


### Dataframes to csv

In [148]:
df_san_francisco_usa_final3.to_csv("../data/API_queries/san_francisco_usa.csv", index = False)

In [149]:
df_new_york_usa_final3.to_csv("../data/API_queries/new_york_usa.csv", index = False)